## Kütüphaneler


In [1]:
pip install EasyDict


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
from tqdm import tqdm
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision.utils as vutils
from torchvision import transforms as trans

from data.ms1m import get_train_loader
from data.lfw import LFW

from backbone.arcfacenet import SEResNet_IR
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint,test



## Konfigürasyon

In [3]:
torch.cuda.empty_cache()
conf=edict()
conf.train_root="./dataset/MS1M"
conf.lfw_test_root="./dataset/lfw_aligned_112"
conf.lfw_file_list="./dataset/lfw_pair.txt"

conf.mode="se_ir" # ir derinlik 100 kullanıyorsak ir modülü daha başarılı sonuç verebilir. Fakat daha düşükte se_ir modülü başarılı
conf.depth=50
conf.margin_type='ArcFace'
conf.feature_dim=512 # çıkacak olan id nin kaça kaçlık olması gerekşiyor
conf.scale_size=32.0
conf.batch_size=16
conf.lr=0.001 #batch size ile doğru orantılı olması gerekiyor batch size düşerse bununda düşmesi lazım
conf.milestones=[8,10,12]
conf.total_epoch=14

conf.save_folder="./saved"
conf.save_dir=os.path.join(conf.save_folder,conf.mode+"_"+str(conf.depth)) #Best modelim /saved/se_ir_100 şeklinde olması gerekiyor


conf.device=torch.device('cuda:0' if torch.cuda.is_available else 'cpu') #eğer cudam varsa cuda ilşe yoksa cpu ile
conf.num_workers=4
conf.pin_memory=True # Daha hızlı Sonuçlar almak için



In [4]:
os.makedirs(conf.save_dir,exist_ok=True)

## Veri Yükleme

In [5]:
transforms=trans.Compose([
    trans.ToTensor(),#Aldığımız resimleri tensor olarak kaydetmemize yardımcı olacak. range[0.255]-> [0.0,1.0]arasında ölçeklendiriyor.
    trans.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))
    
])
trainloader,class_num=get_train_loader(conf)


In [6]:
print(class_num)

200


In [7]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root location: ./dataset/MS1M
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [8]:
lfwdataset=LFW(conf.lfw_test_root,conf.lfw_file_list,transform=transforms)
lfwloader=torch.utils.data.DataLoader(lfwdataset,batch_size=128,num_workers=conf.num_workers)


## Model Oluşturma

In [9]:
print(conf.device) # Nvidia ekran kartına gönderiyorum gpu ile modeli eğitebilirim.

cuda:0


In [10]:
net=SEResNet_IR(conf.depth,feature_dim=conf.feature_dim,mode=conf.mode).to(conf.device)
margin=ArcMarginProduct(conf.feature_dim,class_num).to(conf.device)



In [11]:
print(net)

SEResNet_IR(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): BottleNeck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, moment

In [12]:
criterion=nn.CrossEntropyLoss()


In [13]:
optimizer=optim.SGD([
    {'params':net.parameters(),'weight_decay':5e-4},
    {'params':margin.parameters(),'weight_decay':5e-4},
   
    
],
    lr=conf.lr,
    momentum=0.9,
    nesterov=True ) #gradyanların nasıl hesaplancağını veriyoruz nesterov u kullan dedik

In [14]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


In [15]:
def schedule_lr():
    for params in optimizer.params_groups:
        params['lr'] /=10
    print(optimizer)

## Eğitim

In [ ]:
best_acc=0
for epoch in range(1,conf.total_epoch+1):
    net.train()
    #net.eval() örneğin dropout kullanıyoruz 
    print('epoch {}/{}'.format(epoch,conf.total_epoch),flush=True)
    
    if epoch==conf.milestones[0]:
        schedule_lr()
    if epoch==conf.milestones[1]:
        schedule_lr()
    if epoch==conf.milestones[2]:
        schedule_lr()
    for data in tqdm(trainloader): #eğitimin yapılcağı yer
        img,label=data[0].to(conf.device),data[1].to(conf.device) #datalarımıda gpu ya gönderdim aynı yerde çalışmaları için
        
        optimizer.zero_grad() #gradyanları sıfırladım.
        
        logits=net(img) #Çıktı batch size kadar çıktı alıyorum  
        output=margin(logits,label)
        total_loss=criterion(output,label)
        total_loss.backward()
        optimizer.step()
    #test
    net.eval() #test aşaması değerlendirme modülü
    lfw_acc=test(conf,net,lfwdataset,lfwloader)
    is_best=lfw_acc>best_acc
    best_acc=max(lfw_acc,best_acc)
    #Model kaydetme
    save_checkpoint({
        'epoch':epoch,
        'net_state_dict':net.state_dict(),
        'margin_state_dict':margin.state_dict(),
        'best_acc':best_acc
    },is_best,checkpoint=conf.save_dir)
    
        

epoch 1/14


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [12:51<00:00,  2.36it/s]


best model saved

epoch 2/14


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [12:52<00:00,  2.36it/s]


best model saved

epoch 3/14


...

100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [13:01<00:00,  2.33it/s]


best model saved

epoch 5/14


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [12:56<00:00,  2.35it/s]


epoch 6/14


 16%|████████████▍                                                                  | 288/1822 [02:18<15:06,  1.69it/s]